<a href="https://colab.research.google.com/github/tourihasi/Openstudio/blob/main/ifcopenshell11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install ifcopenshell eppy geomeppy shapely


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.4 MB/s eta 0:00:00


In [ ]:
from google.colab import files

print("IFCファイルを選択してください")
uploaded_ifc = files.upload()

print("Energy+.idd を選択してください")
uploaded_idd = files.upload()


IFCファイルを選択してください


Saving シミュレーション導入.ifc to シミュレーション導入.ifc
Energy+.idd を選択してください


Saving Energy+.idd to Energy+.idd


In [ ]:
# -*- coding: utf-8 -*-
# ================================================================
# [00] IFC -> IDF (Minimal: WALL + WINDOW + FRAME) for EnergyPlus 24.1
#  - 壁(IfcWall/IfcWallStandardCase)を平面統合して BuildingSurface:Detailed (Wall)
#  - Opening→Window(IfcRelVoidsElement / IfcRelFillsElement) を辿って窓を親壁へ割当
#  - 壁からの切り欠きは行わず、窓を親壁に「ぶら下げ」出力（最小版方針）
#  - 公差自動設定・buffer→戻し・重心フォールバックで割当を堅牢化
#  - 4頂点超の窓は三角分割（EnergyPlus 頂点制約回避）
#  - IfcWindowLiningProperties を参照して WindowProperty:FrameAndDivider を生成し、窓へ割当
# ================================================================

# ---------------------------
# [01] Imports & Parameters
# ---------------------------
import os, io
import numpy as np

# 入出力パス（必要に応じて変更）
EPLUS_IDD = "/content/Energy+.idd"   # 本スクリプト自体は参照しません（E+ 実行時の参照用）
IFC_PATH  = "/content/input.ifc"
IDF_OUT   = "/content/out_min_wall_window.idf"

# 既定パラメータ
PLANE_ROUND_BASE    = 5         # 平面キーの丸め（あとで 4 に緩和）
AREA_EPS            = 1e-6      # 面積閾値
ZONE_NAME           = "Zone_1"
SITE                = dict(Name="Tokyo", Latitude=35.68, Longitude=139.76, Time_Zone=9, Elevation=10.0)

# ライニング未設定だった場合の扱い
USE_DEFAULT_FRAME_WHEN_NO_LINING = True   # True: 既定のフレームを自動作成して割当 / False: フレーム無し
DEFAULT_FRAME_WIDTH = 0.05                 # 既定フレーム幅 [m]
DEFAULT_FRAME_COND  = 3.0                  # 既定フレーム熱貫流 [W/m2-K]
DEFAULT_SOLAR_ABS   = 0.6                  # 既定日射吸収率
DEFAULT_VIS_ABS     = 0.6                  # 既定可視吸収率

# 依存
try:
    import ifcopenshell, ifcopenshell.geom as ifc_geom
except Exception as e:
    raise RuntimeError("pip install ifcopenshell") from e
try:
    from shapely.geometry import Polygon, MultiPolygon, Point
    from shapely.ops import unary_union
except Exception as e:
    raise RuntimeError("pip install shapely") from e

# ---------------------------
# [02] IFC Geometry Settings
# ---------------------------
assert os.path.isfile(IFC_PATH), f"IFC not found: {IFC_PATH}"

ifc = ifcopenshell.open(IFC_PATH)
settings = ifc_geom.settings()
def _set_opt(key, val):
    try: settings.set(getattr(settings, key), val)
    except Exception: pass
_set_opt("USE_WORLD_COORDS", True)
_set_opt("INCLUDE_CURVES", False)
_set_opt("DISABLE_TRIANGULATION", False)

# ---------------------------
# [03] Geometry Utilities
# ---------------------------
def normalize(v):
    n = np.linalg.norm(v);  return v/n if n else v

def tri_normal(p0,p1,p2):
    return normalize(np.cross(p1-p0, p2-p0))

def plane_key(p, n, rnd):
    n = normalize(n)
    if n[2] < 0: n = -n
    d = -np.dot(n, p)
    return (round(n[0],rnd), round(n[1],rnd), round(n[2],rnd), round(d,rnd))

def build_plane_basis(n):
    n = normalize(n)
    a = np.array([0,0,1.0]) if abs(n[2]) < 0.9 else np.array([1.0,0,0])
    u = normalize(np.cross(n, a)); v = normalize(np.cross(n, u))
    return u, v

def project2d(P3, origin, u, v):
    P = np.array(P3) - origin
    return np.column_stack([P@u, P@v])

def lift3d(P2, origin, u, v):
    return origin + np.outer(P2[:,0], u) + np.outer(P2[:,1], v)

def shape(el):
    try:
        shp = ifc_geom.create_shape(settings, el)
        V = np.array(shp.geometry.verts, dtype=float).reshape((-1,3))
        F = np.array(shp.geometry.faces, dtype=int).reshape((-1,3))
        return V, F
    except Exception:
        return np.zeros((0,3)), np.zeros((0,3), dtype=int)

# ---------------------------
# [04] Global Tolerances (auto from model extent)
# ---------------------------
def compute_model_tolerance():
    allv = []
    for t in ("IfcWallStandardCase", "IfcWall", "IfcWindow"):
        for el in ifc.by_type(t):
            V,F = shape(el)
            if V.size:
                allv.append(V)
    if not allv:
        return 0.001, PLANE_ROUND_BASE
    V = np.vstack(allv)
    extent = np.ptp(V, axis=0).max()  # 最大寸法
    open_tol = max(0.001, float(extent) * 1e-5)   # 例: 100m → 0.001m
    plane_round = 4  # 少し緩める
    return open_tol, plane_round

OPEN_TOL, PLANE_ROUND = compute_model_tolerance()
print(f"[TOL] OPEN_TOL={OPEN_TOL:.6f}, PLANE_ROUND={PLANE_ROUND}")

# ---------------------------
# [05] Wall Collection & Planar Merge
# ---------------------------
class WallPlane:
    def __init__(self, ifc_id, origin, n, u, v, poly2d):
        self.ifc_id = ifc_id
        self.origin = origin; self.n = n; self.u = u; self.v = v
        self.poly2d = poly2d
        self.windows = []  # list[Polygon2D]

def collect_walls():
    planes = []
    for t in ("IfcWallStandardCase", "IfcWall"):
        for w in ifc.by_type(t):
            V,F = shape(w)
            if V.size==0 or F.size==0: continue
            groups={}
            for (a,b,c) in F:
                p0,p1,p2 = V[a],V[b],V[c]
                pk = plane_key(p0, tri_normal(p0,p1,p2), PLANE_ROUND)
                groups.setdefault(pk, []).append(np.vstack([p0,p1,p2]))
            for tris in groups.values():
                tris = np.array(tris)
                origin = tris[0,0,:]
                n = tri_normal(tris[0,0,:], tris[0,1,:], tris[0,2,:])
                if n[2] < 0: n = -n
                u,v = build_plane_basis(n)
                polys=[]
                for tri in tris:
                    p2 = Polygon(project2d(tri, origin,u,v))
                    if not p2.is_valid: p2 = p2.buffer(0)
                    if p2.area > AREA_EPS: polys.append(p2)
                if not polys: continue
                merged = unary_union(polys)
                geoms = [merged] if isinstance(merged,Polygon) else list(merged.geoms)
                for g in geoms:
                    if g.area > AREA_EPS:
                        planes.append(WallPlane(w.id(), origin, n, u, v, Polygon(g.exterior.coords)))
    print(f"[Walls] Collected planes: {len(planes)}")
    return planes

# ---------------------------
# [06] Opening→Window Relations
# ---------------------------
def build_maps():
    open_to_wall = {}
    for rel in ifc.by_type("IfcRelVoidsElement"):
        be = getattr(rel,"RelatingBuildingElement",None)
        oe = getattr(rel,"RelatedOpeningElement",None)
        if be and oe:
            open_to_wall[oe.id()] = be.id()

    open_to_windows = {}
    for rel in ifc.by_type("IfcRelFillsElement"):
        oe = getattr(rel,"RelatingOpeningElement",None)
        be = getattr(rel,"RelatedBuildingElement",None)
        if oe and be and be.is_a()=="IfcWindow":
            open_to_windows.setdefault(oe.id(), []).append(be)
    return open_to_wall, open_to_windows

# ---------------------------
# [07] Assign Windows to Wall Planes (robust)
# ---------------------------
def assign_windows(walls, open_to_wall, open_to_windows):
    # 壁インデックス（ifc_id→list[WallPlane]）
    idx = {}
    for wp in walls:
        idx.setdefault(wp.ifc_id, []).append(wp)

    def window_2d_on_wp(winV, winF, wp):
        polys=[]
        for (a,b,c) in winF:
            tri = np.vstack([winV[a],winV[b],winV[c]])
            p2  = Polygon(project2d(tri, wp.origin, wp.u, wp.v))
            if not p2.is_valid: p2 = p2.buffer(0)
            if p2.area > AREA_EPS: polys.append(p2)
        if not polys:
            return None
        merged = unary_union(polys)
        # 微小隙間を吸収
        merged = merged.buffer(OPEN_TOL).buffer(-OPEN_TOL)
        return merged

    for open_id, win_list in open_to_windows.items():
        host = open_to_wall.get(open_id)
        if not host or host not in idx:
            continue
        for win in win_list:
            V,F = shape(win)
            if V.size==0 or F.size==0: continue
            best_wp, best_area, best_poly = None, 0.0, None
            for wp in idx[host]:
                merged = window_2d_on_wp(V, F, wp)
                if merged is None:
                    continue
                # 壁側も buffer→戻し
                wall2d = wp.poly2d.buffer(OPEN_TOL).buffer(-OPEN_TOL)
                inter  = wall2d.intersection(merged)
                area   = inter.area
                if area > best_area:
                    best_area = area
                    best_wp   = wp
                    best_poly = inter if inter.area > AREA_EPS else merged
            # フォールバック：重心が壁の buffer 内なら採用
            if (not best_wp or best_area <= AREA_EPS) and host in idx:
                merged_any = None
                for wp in idx[host]:
                    tmp = window_2d_on_wp(V, F, wp)
                    if tmp is not None:
                        merged_any = tmp; break
                if merged_any is not None:
                    c = merged_any.centroid
                    for wp in idx[host]:
                        if wp.poly2d.buffer(max(OPEN_TOL,0.002)).contains(c):
                            best_wp = wp; best_poly = merged_any; break
            # 採用
            if best_wp and best_poly and best_poly.area > AREA_EPS:
                if isinstance(best_poly, MultiPolygon):
                    for g in best_poly.geoms:
                        if g.area > AREA_EPS:
                            best_wp.windows.append(Polygon(g.exterior.coords))
                else:
                    best_wp.windows.append(Polygon(best_poly.exterior.coords))

# ---------------------------
# [08] IFC Window Lining -> FrameAndDivider
# ---------------------------
def get_window_type(win):
    # IfcWindow -> IfcWindowType（IsTypedBy 経由）
    for rel in getattr(win, "IsTypedBy", []):
        if rel and rel.is_a("IfcRelDefinesByType"):
            if rel.RelatingType and rel.RelatingType.is_a("IfcWindowType"):
                return rel.RelatingType
    return None

def get_window_lining_from_type(wtype):
    # 型の HasPropertySets から IfcWindowLiningProperties を探す
    for pset in getattr(wtype, "HasPropertySets", []) or []:
        if pset and pset.is_a("IfcWindowLiningProperties"):
            return pset
    # IFC2x3 などでは別経路のこともあるが、最小版ではここまで
    return None

def lining_to_frame_spec(lining):
    """IfcWindowLiningProperties -> dict(EP frame spec). 未設定は None に."""
    def val(x):
        return None if x in (None, "", "$") else float(x)
    return dict(
        width = val(getattr(lining, "LiningThickness", None)),
        out_proj = val(getattr(lining, "LiningOffset", None)) or 0.0,  # 便宜上 offset を外側投影に
        in_proj  = 0.0,
        cond     = DEFAULT_FRAME_COND,
        solar_abs= DEFAULT_SOLAR_ABS,
        vis_abs  = DEFAULT_VIS_ABS,
        # ここでは Divider は使わない（None）
    )

# 窓→フレーム名の割当管理（重複生成防止）
FRAME_REGISTRY = {}  # key: tuple(values) -> frame_name
FRAME_USED_BY  = {}  # key: frame_name -> count

def frame_key_from_spec(spec):
    return (round(spec["width"] or 0.0, 6),
            round(spec["out_proj"], 6),
            round(spec["in_proj"], 6),
            round(spec["cond"], 6),
            round(spec["solar_abs"], 6),
            round(spec["vis_abs"], 6))

def ensure_frame_name_for_window(win):
    """IFC窓からライニングを取り、FrameAndDivider 名を返す（必要なら既定値）"""
    wtype = get_window_type(win)
    lining = get_window_lining_from_type(wtype) if wtype else None
    if lining:
        spec = lining_to_frame_spec(lining)
    else:
        # ライニングが見つからない／全て未設定のケース
        if not USE_DEFAULT_FRAME_WHEN_NO_LINING:
            return ""  # 割当しない
        spec = dict(
            width=DEFAULT_FRAME_WIDTH, out_proj=0.0, in_proj=0.0,
            cond=DEFAULT_FRAME_COND, solar_abs=DEFAULT_SOLAR_ABS, vis_abs=DEFAULT_VIS_ABS
        )

    # 幅が 0/None の場合の扱い
    w = spec.get("width", None)
    if (w is None or w <= 0.0):
        if USE_DEFAULT_FRAME_WHEN_NO_LINING:
            spec["width"] = DEFAULT_FRAME_WIDTH
        else:
            return ""  # 幅ゼロならフレーム未割当

    key = frame_key_from_spec(spec)
    if key in FRAME_REGISTRY:
        name = FRAME_REGISTRY[key]
    else:
        name = f"Frame_{len(FRAME_REGISTRY)+1}"
        FRAME_REGISTRY[key] = name
    FRAME_USED_BY[name] = FRAME_USED_BY.get(name, 0) + 1
    return name, spec

# ---------------------------
# [09] IDF Helpers (E+ 24.1 compatible)
# ---------------------------
def fmt_num(x):
    return f"{x:.6f}".rstrip('0').rstrip('.') if isinstance(x, float) else str(x)

def idf_line(fields, trailing_semicolon=True):
    s = fields[0] + ",\n"
    for i,(val,comment) in enumerate(fields[1:], start=1):
        comma = ";" if (i==len(fields)-1 and trailing_semicolon) else ","
        s += f"  {val}{comma}" + (f"  !- {comment}\n" if comment else "\n")
    return s

def idf_header():
    out = ""
    out += idf_line(["Version", ("24.1","")])
    out += idf_line(["Site:Location",
                     (SITE["Name"],"Name"),
                     (fmt_num(SITE["Latitude"]),"Latitude"),
                     (fmt_num(SITE["Longitude"]),"Longitude"),
                     (fmt_num(SITE["Time_Zone"]),"Time Zone"),
                     (fmt_num(SITE["Elevation"]),"Elevation")])
    out += idf_line(["Timestep", ("6","Number of Timesteps per Hour")])
    out += idf_line(["Building",
                     ("IFC_Min_Wall_Window","Name"),
                     ("0.0","North Axis"),
                     ("City","Terrain"),
                     ("0.04","Loads Convergence Tolerance Value"),
                     ("0.4","Temperature Convergence Tolerance Value"),
                     ("FullInteriorAndExteriorWithReflections","Solar Distribution"),
                     ("25","Maximum Number of Warmup Days"),
                     ("6","Minimum Number of Warmup Days")])
    out += idf_line(["Zone", (ZONE_NAME,"Name")])
    # Constructions/Materials
    out += idf_line(["


Total IfcSpace: 14
Triangles(before): 486  ->  Merged planar surfaces(after): 167
Saved: /content/out_merged.idf
